In [8]:
import kagglehub

path = kagglehub.dataset_download("mahmoudreda55/satellite-image-classification")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/satellite-image-classification


In [9]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import os

In [10]:
desert_folder = '/kaggle/input/satellite-image-classification/data/desert'
augmented_folder = '/kaggle/working/Desert_Augmented'
os.makedirs(augmented_folder, exist_ok=True)

In [11]:
all_images = sorted(os.listdir(desert_folder))
images_to_augment = all_images[::2][:369]

In [12]:
def rotate_image(image, angle):
    h, w = image.shape[:2]
    center = (w // 2, h // 2)
    rotation_matrix = cv2.getRotationMatrix2D(center, angle, 1.0)
    rotated_image = cv2.warpAffine(image, rotation_matrix, (w, h))
    return rotated_image

def flip_image(image, horizontal=True):
    
    if horizontal:
        return cv2.flip(image, 1)
    else:
        return cv2.flip(image, 0)

for idx, img_name in enumerate(images_to_augment):
    img_path = os.path.join(desert_folder, img_name)
    
    image = cv2.imread(img_path)
    if image is None:
        print(f"Warning: Couldn't read {img_path}")
        continue
    
    rotated_image = rotate_image(image, angle=90)
    flipped_image = flip_image(rotated_image, horizontal=True)

    
    #cv2.imwrite(os.path.join(augmented_folder, f"desert_rotated_{idx}.jpg"), rotated_image)
    cv2.imwrite(os.path.join(augmented_folder, f"desert_flipped_{idx}.jpg"), flipped_image)

print("Augmentation completed. Augmented images saved to:", augmented_folder)

Augmentation completed. Augmented images saved to: /kaggle/working/Desert_Augmented


In [15]:
import os
import shutil
from sklearn.model_selection import train_test_split
import cv2

input_base_path = '/kaggle/input/satellite-image-classification/data'
augmented_path = '/kaggle/working/Desert_Augmented'
output_base_path = '/kaggle/working/prepared_dataset' 

categories = ['cloudy', 'desert', 'green_area', 'water']
split_ratios = {'train': 0.7, 'val': 0.15, 'test': 0.15}

def copy_images(image_list, source_folder, target_folder, augmented_images=None):

    for img in image_list:
        if augmented_images and img in augmented_images:
            src = os.path.join(augmented_path, img)
        else:
            src = os.path.join(source_folder, img)
        
        dest = os.path.join(target_folder, img)
        shutil.copy(src, dest)


In [16]:
for split in ['train', 'val', 'test']:
    for category in categories:
        os.makedirs(os.path.join(output_base_path, split, category), exist_ok=True)


for category in categories:
    if category == 'desert':
        desert_original = sorted(os.listdir(os.path.join(input_base_path, 'desert')))
        desert_augmented = sorted(os.listdir(augmented_path))
        all_images = desert_original + desert_augmented
    else:
        all_images = sorted(os.listdir(os.path.join(input_base_path, category)))
    
    
    train_images, temp_images = train_test_split(all_images, test_size=split_ratios['val'] + split_ratios['test'], random_state=42)
    val_images, test_images = train_test_split(temp_images, test_size=split_ratios['test'] / (split_ratios['val'] + split_ratios['test']), random_state=42)
    
    
    for split, image_list in zip(['train', 'val', 'test'], [train_images, val_images, test_images]):
        split_folder = os.path.join(output_base_path, split, category)
        copy_images(image_list, os.path.join(input_base_path, category), split_folder, desert_augmented if category == 'desert' else None)

print("Dataset preparation complete. Folders created at:", output_base_path)

for split in ['train', 'val', 'test']:
    print(f"\nChecking {split} data:")
    for category in os.listdir(os.path.join(output_base_path, split)):
        category_path = os.path.join(output_base_path, split, category)
        num_images = len(os.listdir(category_path))
        print(f"  {category}: {num_images} images")

Dataset preparation complete. Folders created at: /kaggle/working/prepared_dataset

Checking train data:
  green_area: 1050 images
  desert: 1050 images
  cloudy: 1050 images
  water: 1050 images

Checking val data:
  green_area: 225 images
  desert: 225 images
  cloudy: 225 images
  water: 225 images

Checking test data:
  green_area: 225 images
  desert: 225 images
  cloudy: 225 images
  water: 225 images


In [22]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(256, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(4, activation='softmax')
])

model.compile(optimizer=Adam(0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)                    │ (None, 148, 148, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_3 (MaxPooling2D)       │ (None, 74, 74, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_4 (Conv2D)                    │ (None, 72, 72, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_4 (MaxPooling2D)       │ (None, 36, 36, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_5 (Conv2D)                    │ (None, 34, 34, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_5 (MaxPooling2D)       │ (None, 17, 17, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_6 (Conv2D)                    │ (None, 15, 15, 256)         │         295,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_6 (MaxPooling2D)       │ (None, 7, 7, 256)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_1 (Flatten)                  │ (None, 12544)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 256)                 │       3,211,520 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 4)                   │           1,028 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 3,600,964 (13.74 MB)

 Trainable params: 3,600,964 (13.74 MB)

 Non-trainable params: 0 (0.00 B)

In [23]:
train_dir = '/kaggle/working/prepared_dataset/train'
val_dir = '/kaggle/working/prepared_dataset/val'

train_dataset = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    image_size=(150, 150),
    batch_size=32,
    label_mode='categorical'
)

val_dataset = tf.keras.utils.image_dataset_from_directory(
    val_dir,
    image_size=(150, 150),
    batch_size=32,
    label_mode='categorical'
)

Found 4200 files belonging to 4 classes.
Found 900 files belonging to 4 classes.


In [24]:
early_stopping = EarlyStopping(
    monitor='val_accuracy',    
    patience=5,                
    mode='max',                
    restore_best_weights=True  
)

# Train the model with EarlyStopping
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=50,                
    callbacks=[early_stopping]
)
# Save the model
model.save('/kaggle/working/satellite_model_simple.h5')
print("Model saved successfully!")

Epoch 1/100
132/132 ━━━━━━━━━━━━━━━━━━━━ 11s 51ms/step - accuracy: 0.6944 - loss: 2.3970 - val_accuracy: 0.8778 - val_loss: 0.2571
Epoch 2/100
132/132 ━━━━━━━━━━━━━━━━━━━━ 3s 24ms/step - accuracy: 0.8642 - loss: 0.2933 - val_accuracy: 0.9189 - val_loss: 0.1880
Epoch 3/100
132/132 ━━━━━━━━━━━━━━━━━━━━ 3s 25ms/step - accuracy: 0.8794 - loss: 0.2790 - val_accuracy: 0.8922 - val_loss: 0.2458
Epoch 4/100
132/132 ━━━━━━━━━━━━━━━━━━━━ 3s 25ms/step - accuracy: 0.8908 - loss: 0.2395 - val_accuracy: 0.9100 - val_loss: 0.1916
Epoch 5/100
132/132 ━━━━━━━━━━━━━━━━━━━━ 3s 25ms/step - accuracy: 0.9159 - loss: 0.2010 - val_accuracy: 0.8989 - val_loss: 0.2039
Epoch 6/100
132/132 ━━━━━━━━━━━━━━━━━━━━ 3s 26ms/step - accuracy: 0.9076 - loss: 0.2165 - val_accuracy: 0.9122 - val_loss: 0.1802
Epoch 7/100
132/132 ━━━━━━━━━━━━━━━━━━━━ 3s 25ms/step - accuracy: 0.9035 - loss: 0.2123 - val_accuracy: 0.8978 - val_loss: 0.2084
Epoch 8/100
132/132 ━━━━━━━━━━━━━━━━━━━━ 3s 25ms/step - accuracy: 0.9291 - loss: 0.1779 -

In [25]:
test_dir = '/kaggle/working/prepared_dataset/test'

test_dataset = tf.keras.utils.image_dataset_from_directory(
    test_dir,
    image_size=(150, 150),
    batch_size=32,
    label_mode='categorical'
)

test_loss, test_accuracy = model.evaluate(test_dataset)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

Found 900 files belonging to 4 classes.
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.9798 - loss: 0.1644
Test Accuracy: 98.44%
